## Оценка эффекта от переезда в большой город на зарплату

In [3]:
import pandas as pd
from scipy import stats
import numpy as np 

In [4]:
df = pd.read_excel('wage_gap.xlsx')

In [5]:
df.columns

Index(['n', 't', 'children', 'fam_size', 'hours', 'married', 'region',
       'av_central_SMSA', 'SMSA_central', 'not_central_SMSA', 'SMSA_not',
       'urban', 'wage', 'cpi_w', 'years', 'AFQT2', 'black', 'class_of_work',
       'education', 'HGT_father', 'HGT_mother', 'hispanic', 'promotion',
       'risk', 'sample_id_79', 'self_conf', 'shiblings', 'size_of_firm',
       'south_birth', 'union', 'town_14', 'white', 'woman'],
      dtype='object')

In [15]:
df

,n,t,children,fam_size,hours,married,region,av_central_SMSA,SMSA_central,not_central_SMSA,...,risk,sample_id_79,self_conf,shiblings,size_of_firm,south_birth,union,town_14,white,woman
0,222,1979,0.0,11.0,0.000000,0.0,1.0,0,1,0,...,1.0,3,16.0,6.0,2.0,1.0,NaN,1,0,0
1,222,1980,0.0,11.0,25.000000,0.0,1.0,0,1,0,...,1.0,3,16.0,6.0,2.0,1.0,NaN,1,0,0
2,222,1981,0.0,11.0,60.000000,0.0,1.0,0,1,0,...,1.0,3,16.0,6.0,2.0,1.0,NaN,1,0,0
3,222,1982,0.0,12.0,0.000000,0.0,1.0,0,1,0,...,1.0,3,16.0,6.0,2.0,1.0,NaN,1,0,0
4,222,1983,0.0,12.0,0.000000,0.0,1.0,0,1,0,...,1.0,3,16.0,6.0,2.0,1.0,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27531,6137,1990,0.0,7.0,3064.000000,0.0,3.0,0,1,0,...,0.0,10,22.0,8.0,150.0,1.0,1.0,1,0,0
27532,6137,1991,0.0,6.0,160.000000,0.0,3.0,0,1,0,...,0.0,10,22.0,8.0,150.0,1.0,1.0,1,0,0
27533,6137,1992,0.0,1.0,2192.000000,0.0,3.0,0,1,0,...,0.0,10,22.0,8.0,150.0,1.0,1.0,1,0,0
27534,6137,1993,0.0,1.0,1479.444358,0.0,3.0,0,1,0,...,0.0,10,22.0,8.0,150.0,1.0,1.0,1,0,0


In [32]:
# Оставим самые логичные для анализа колонки 
cols = selected_columns = [
    'n',
    't',
    'cpi_w',  
    'SMSA_central',
    'not_central_SMSA',  
    'av_central_SMSA',  
    'class_of_work',
    'black',
    'education',
    'size_of_firm',
    'SMSA_not',
    'region', 
    'hours',  
    'years',
    'fam_size',
    'married',  
    'white',  
    'woman',  
    'children',
    'town_14'
]
df = df[cols]
# удалим записи с неизвестной зарплатой
df = df[df['cpi_w'].isna()==False]
df2 = pd.DataFrame()
for _, data in df.groupby('n'):
    data = data.sort_values('t')
    data = data.fillna(method='ffill')
    df2 = pd.concat([df2,data])
df = df2
#Оставим людей от 16 лет
df = df[df['years']>=16]
df['education'] = df['education'].astype(int)

C:\Users\marin\AppData\Local\Temp\ipykernel_35936\1211410141.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


In [33]:
#добавим индикатор года перезда в другой город
df['move_year'] = 0
for person_id, group in df.groupby('n'):
    group = group.sort_values('t')
    move_index = group[(group['SMSA_central'] == 1) & (group['SMSA_central'].shift(1) == 0)].index
    if not move_index.empty:
        df.loc[move_index, 'move_year'] = 1

## Выделим переехавших и всегда живших в маленьком городе
moved = df[df['n'].isin(df[df['move_year']==1]['n'])]
stayed = df[~df['n'].isin(df[df['SMSA_central']==1]['n'])][df['n'].isin(df[df['SMSA_central']==0]['n'])]

C:\Users\marin\AppData\Local\Temp\ipykernel_35936\1684203166.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stayed = df[~df['n'].isin(df[df['SMSA_central']==1]['n'])][df['n'].isin(df[df['SMSA_central']==0]['n'])]


In [110]:
moved[moved['SMSA_central']==1]['cpi_w'].mean() -  stayed['cpi_w'].mean()

-444.7452809354904

In [35]:
moved[moved['SMSA_central']==1]['cpi_w'].mean(), moved[moved['SMSA_central']==0]['cpi_w'].mean()

(1164.2347857286313, 723.2448132562)

In [ ]:
moved['treat'] = 1
stayed['treat'] = 0

In [111]:
variables =     [  
    'class_of_work',
    'black',
    'education',
    'size_of_firm',
    'region', 
    'hours',  
    'years',
    'fam_size',
    'married',  
    'white',  
    'woman',  
    'children',
    'town_14']

In [112]:
# Для категориальных факторов используем Критерий хи квадрат по частотам, для непрерывных тест Уэлча 
pd.set_option('display.float_format', lambda x: '%.4f' % x)
def calculate_balance_table(df, group_var, variables):
    balance_table = []

    for var in variables:
        mean_treat = df[df[group_var] == 1][var].mean()
        std_treat = df[df[group_var] == 1][var].std()
        std_control = df[df[group_var] == 0][var].std()
        mean_control = df[df[group_var] == 0][var].mean()
        diff = mean_treat - mean_control

        std_error = stats.sem(df[df[group_var] == 1][var].dropna()) + stats.sem(df[df[group_var] == 0][var].dropna())

        balance_table.append({
            'Variable': var,
            'Mean (Treatment)': mean_treat,
            'Mean (Control)': mean_control,
            'Std (Treatment)':std_treat,
            'Std (Control)':std_control,
            'Difference': diff,
            'Std. Error': std_error,
            'Observations (Treatment)': df[df[group_var] == 1][var].count(),
            'Observations (Control)': df[df[group_var] == 0][var].count()
        })
    balance_table = pd.DataFrame(balance_table)
    for ind, row in balance_table.iterrows():
        if (row['Variable'] != 'cpi_w' and row['Variable'] != 'size_of_firm' and row['Variable'] != 'hours'):
            sample1 = df[df['treat']==1][row['Variable']].dropna().astype(int).to_list()
            sample2 = df[df['treat']==0][row['Variable']].dropna().astype(int).to_list()
            unique_values = sorted(set(sample1).union(set(sample2)))
            sample1 = [sample1.count(value) / len(sample1) * 100 for value in unique_values]
            sample2 = [sample2.count(value) / len(sample2) * 100 for value in unique_values]
            table = [sample1,
                    sample2]
            
            p_value = stats.chi2_contingency(table)[1]
            p_value = float(p_value)
            balance_table.at[ind,'тест'] = 'Хи 2'
            balance_table.at[ind,'p-val'] = p_value

        else:
            sample1 = df[df['treat']==1][row['Variable']].dropna().astype(float).to_list()
            sample2 = df[df['treat']==0][row['Variable']].dropna().astype(float).to_list()
            t_stat,p_value = stats.ttest_ind(sample1, sample2, alternative='two-sided')
            p_value = float(p_value)
            balance_table.at[ind,'тест'] = 'Уэлча'
            balance_table.at[ind,'p-val'] = p_value
    return balance_table
balance_tab = calculate_balance_table(pd.concat([stayed,moved[moved['SMSA_central']==1]]),'treat', variables)

In [113]:
balance_tab

,Variable,Mean (Treatment),Mean (Control),Std (Treatment),Std (Control),Difference,Std. Error,Observations (Treatment),Observations (Control),тест,p-val
0,class_of_work,1.1948,1.2184,0.4073,0.5268,-0.0236,0.0230,462,16760,Хи 2,0.0479
1,black,0.4913,0.2641,0.5005,0.4409,0.2272,0.0267,462,16805,Хи 2,0.0015
2,education,13.9589,12.4031,2.2483,1.8176,1.5558,0.1186,462,16805,Хи 2,0.0279
3,size_of_firm,4445.7222,3782.6500,16097.5556,18271.9162,663.0722,901.7875,450,16340,Уэлча,0.4462
4,region,2.4091,2.6114,1.0699,0.9320,-0.2023,0.0570,462,16791,Хи 2,0.1201
5,hours,1931.4806,1739.8069,857.1498,972.5685,191.6737,47.4239,462,16613,Уэлча,0.0000
6,years,28.3420,24.6587,3.7038,4.7656,3.6833,0.2091,462,16805,Хи 2,0.0466
7,fam_size,2.6797,3.6849,1.5563,1.9212,-1.0052,0.0872,462,16805,Хи 2,0.1392
8,married,0.5260,0.5088,0.4999,0.4999,0.0172,0.0271,462,16804,Хи 2,0.9192
9,white,0.3636,0.5805,0.4816,0.4935,-0.2169,0.0262,462,16805,Хи 2,0.0034


# Мэтчинг

In [121]:
# Тут просто решение, где мы находим для каждого переехвшего ниболее похожего их оставшихся
from sklearn.neighbors import NearestNeighbors
covars = variables =[  
    'class_of_work',
    'black',
    'education',
    'size_of_firm',
    'region', 
    'hours',  
    'years',
    'fam_size',
    'married',  
    'white',  
    'woman',  
    'children',
    'town_14']
control = stayed[['n','t','cpi_w','treat','move_year']+covars].copy().dropna()
treated = moved[moved['SMSA_central']==1][['n','t','cpi_w','treat','move_year']+covars].copy().dropna()


nn = NearestNeighbors(n_neighbors=1)
nn.fit(control[covars])
distances, indices = nn.kneighbors(treated[covars])

matched_controls = control.iloc[indices[distances<10].flatten()]
#matched_controls = control.iloc[indices[distances<10].flatten()]
treated['cpi_w'].mean() - matched_controls['cpi_w'].mean()

199.03879186487177

In [120]:
# Функция расчета среднего эффекта
def mean_effect(data, treatment_var, score_var):
    mean_treat = data[data[treatment_var] == 1][score_var].mean()
    mean_control = data[data[treatment_var] == 0][score_var].mean()
    return mean_treat - mean_control

# Функция по созданию бутсрап выборки и расчета интервалов 
def bootstrap_self(data, treatment_var, score_var, n_bootstrap=5000, confidence_level=0.95):
    bootstrap_estimates = []
    bootstrap_estimates_hall = []
    bootstrap_estimates_t_proc = []
    for _ in range(n_bootstrap):
        bootstrap_sample = data.sample(frac=1, replace=True)
        effect = mean_effect(bootstrap_sample, treatment_var, score_var)
        eff_hall = effect - mean_effect(data,treatment_var, score_var) 
        eff_t_proc = (effect - mean_effect(data,treatment_var, score_var)) / bootstrap_sample[score_var].std()
        bootstrap_estimates_t_proc.append(eff_t_proc)
        bootstrap_estimates.append(effect)
        bootstrap_estimates_hall.append(eff_hall)
    
    effect = np.mean(bootstrap_estimates)
    lower_bound = np.percentile(bootstrap_estimates, (1 - confidence_level) / 2 * 100) 
    upper_bound = np.percentile(bootstrap_estimates, (1 + confidence_level) / 2 * 100) 
    
    return effect,(lower_bound, upper_bound)

mean_eff, ci = bootstrap_self(pd.concat([treated, matched_controls]), 'treat', 'cpi_w')


results = {
    'Mean Effect': [mean_eff],
    'Нижний доверительный интервал': [ci[0]],
    'Верхний доверительный интервал': [ci[1]],
}

results = pd.DataFrame(results)
results

,Mean Effect,Нижний доверительный интервал,Верхний доверительный интервал
0,197.3274,55.7254,321.1365


# Оценка эффекта после процедуры мэтчинга

In [119]:
#Стало чуть лучше, но незначительно
balance = calculate_balance_table(pd.concat([treated, matched_controls]),'treat',covars)
balance

,Variable,Mean (Treatment),Mean (Control),Std (Treatment),Std (Control),Difference,Std. Error,Observations (Treatment),Observations (Control),тест,p-val
0,class_of_work,1.2000,1.1763,0.4114,0.4340,0.0237,0.0433,450,329,Хи 2,0.3305
1,black,0.5044,0.2432,0.5005,0.4296,0.2613,0.0473,450,329,Хи 2,0.0002
2,education,14.0111,13.0517,2.2549,1.7705,0.9594,0.2039,450,329,Хи 2,0.0490
3,size_of_firm,4445.7222,3412.5137,16097.5556,17146.1202,1033.2085,1704.1423,450,329,Уэлча,0.3896
4,region,2.3933,2.5532,1.0797,0.9195,-0.1599,0.1016,450,329,Хи 2,0.0799
5,hours,1937.3490,1895.9453,852.4867,700.5690,41.4037,78.8102,450,329,Уэлча,0.4713
6,years,28.3378,27.8024,3.7102,3.5980,0.5353,0.3733,450,329,Хи 2,0.9984
7,fam_size,2.6556,3.0851,1.5319,1.4095,-0.4296,0.1499,450,329,Хи 2,0.2574
8,married,0.5133,0.6565,0.5004,0.4756,-0.1432,0.0498,450,329,Хи 2,0.0559
9,white,0.3467,0.6505,0.4764,0.4776,-0.3038,0.0488,450,329,Хи 2,0.0000


In [138]:
covars = variables =[  
    'class_of_work',
    'black',
    'education',
    'size_of_firm',
    'region', 
    'hours',  
    'years',
    'fam_size',
    'married',  
    'white',  
    'woman',  
    'children',
    'town_14']
control = stayed[['n','t','treat','move_year','cpi_w','SMSA_central']+covars].copy().dropna()
treated = moved[['n','t','treat','move_year','cpi_w','SMSA_central']+covars].copy().dropna()


In [139]:
#Оставим по каждому человеку одну запись. Для уехавших -- это год переезда, для остальных -- первая запись 
import statsmodels.api as sm
move_dat = pd.concat([treated[treated['move_year']==1], control.drop_duplicates(subset=['n'],keep='first')])
logit_model = sm.Logit(move_dat['move_year'], sm.add_constant(move_dat[covars])).fit()
display(logit_model.aic)
prob_model = sm.Probit(move_dat['move_year'], sm.add_constant(move_dat[covars])).fit()
display(prob_model.aic)

Optimization terminated successfully.
         Current function value: 0.029206
         Iterations 11


99.08803322901576

Optimization terminated successfully.
         Current function value: 0.031315
         Iterations 11


104.22120313215342

In [140]:
# Оставим пробит по aic критерию
data = pd.concat([treated,control])
data['pros_score'] = logit_model.predict(sm.add_constant(data[covars]))

In [141]:
# Доавим к каждой записи вероятность перезда в этот момент времени
# Для контрольной группы посчитаем разницу средней зарплаты до и после прогноза переезда
control = data[data['treat']==0].copy().reset_index(drop=True)
control_pros = []
for n,person_data in control.groupby('n'):
    person_data = person_data.sort_values('t')
    for ind, row in person_data.iterrows():
        if row['pros_score'] > 0.5:
            cpi_w = person_data [person_data ['t']>=row['t']]['cpi_w'].mean() - person_data [person_data ['t']<row['t']]['cpi_w'].mean()
            control_pros.append({
                'n': n,
                'cpi_w': cpi_w,
                'score': row['pros_score'],
                'treatment': 0
            })
            break
control = pd.DataFrame(control_pros)
treat = data[data['treat']==1].copy().reset_index(drop=True)

treat_pros = []
for n, person_data in treat.groupby('n'):
    for ind, row in person_data.iterrows():
        if row['move_year']==1:
            cpi_w = person_data[person_data['SMSA_central']==1]['cpi_w'].mean() - person_data[person_data['SMSA_central']==0]['cpi_w'].mean()
            treat_pros.append({
                'n': n,
                'cpi_w': cpi_w,
                'score': row['pros_score'],
                'treatment': 1
            })
            break
treat = pd.DataFrame(treat_pros)

In [147]:
data

,n,cpi_w,score,treatment,weights
0,184,332.6166,1.0000,1,1.0000
1,231,261.7295,1.0000,1,1.0000
2,483,-1.3863,0.9172,1,1.0902
3,847,710.4094,0.9987,1,1.0013
4,850,366.0764,1.0000,1,1.0000
...,...,...,...,...,...
1210,12478,220.6320,0.8936,0,9.4027
1211,12479,127.7874,0.5268,0,2.1134
1212,12515,561.9226,0.9978,0,456.9649
1213,12517,101.0310,0.6012,0,2.5072


In [148]:
data.rename(columns={'cpi_w':'Прирост зп после переезда','score':'propensity score','treatment':'Перехавший'})

,n,Прирост зп после переезда,propensity score,Перехавший,weights
0,184,332.6166,1.0000,1,1.0000
1,231,261.7295,1.0000,1,1.0000
2,483,-1.3863,0.9172,1,1.0902
3,847,710.4094,0.9987,1,1.0013
4,850,366.0764,1.0000,1,1.0000
...,...,...,...,...,...
1210,12478,220.6320,0.8936,0,9.4027
1211,12479,127.7874,0.5268,0,2.1134
1212,12515,561.9226,0.9978,0,456.9649
1213,12517,101.0310,0.6012,0,2.5072


In [164]:
from statsmodels.regression.linear_model import WLS
data = pd.concat([treat,control]).dropna().reset_index(drop=True)
data['weights'] = np.where(
    data['treatment'] == 1,
    1 / data['score'],  
    1 / (1 - data['score'])  
)
wls_model = WLS(data['cpi_w'], sm.add_constant(data['treatment']), weights=data['weights']).fit()

In [165]:
wls_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  cpi_w   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3099
Date:                Mon, 25 Nov 2024   Prob (F-statistic):              0.578
Time:                        01:08:34   Log-Likelihood:                -14342.
No. Observations:                1215   AIC:                         2.869e+04
Df Residuals:                    1213   BIC:                         2.870e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.5943    949.978      0.051      0.959   -1815.188    1912.376
treatment    703.7275   1264.156      0.557      0.578   -1776.448    3183.903
==============================================================================
Omnibus:                     3200.615   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         49161926.814
Skew:                         -28.981   Prob(JB):                         0.00
Kurtosis:                     986.738   Cond. No.                         2.80
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""